#### 1. 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

train_path= 'C:/Users/82109/Desktop/파일/Dataset/Titanic/train.csv'
test_path= 'C:/Users/82109/Desktop/파일/Dataset/Titanic/test.csv'


train= pd.read_csv(train_path, header= 0, dtype= {'Age':np.float64})
test= pd.read_csv(test_path, header= 0, dtype= {'Age':np.float64})
full_data= [train, test]

#### 2. 특성 공학(Feature Engineering)

1. Pclass : 좌석 등급<br>
- 결측치 없다.
- Pclass에 따른 타겟(Survived)의 값을 확인한다.

In [9]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index= False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


2. Sex: 성별 <br>
- Pclass에 따른 타겟(Survived)의 값을 확인한다.

In [10]:
train[['Sex','Survived']].groupby(['Sex'], as_index= False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


3. SibSp and Parch: 형제/자매, 자녀/부모
- 결측치 없음
- 이 둘의 합으로 FamilySize 라는 새로운 특성을 만들 수 있다.
- FamilySize에 따른 Survived의 평균
- 승객이 혼자인지 아닌지를 IsAlone 라는 새로운 특성을 만든다.
- IsAlone에 따른 Survived 평균을 구한다.

In [13]:
# FamilySize
for dataset in full_data:
    dataset['FamilySize']= dataset['SibSp'] + dataset['Parch'] + 1 # 1은 보인

train[['FamilySize', 'Survived']].groupby(['FamilySize'],as_index= False).mean()

,FamilySize,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


In [17]:
# IsAlone
for dataset in full_data:
    dataset['IsAlone']= 0
    dataset.loc[dataset['FamilySize']== 1, 'IsAlone']= 1

train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index= False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


4. Embarked: 탑승지
- 결측치 있음: 가장 많은 S로 채우겠음

In [19]:
for dataset in full_data:
    dataset['Embarked']= dataset['Embarked'].fillna('S')

train[['Embarked', 'Survived']].groupby(['Embarked'], as_index= False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


5. Fare: 비용
- 결측치 있음: 중간값으로 채운다.
- 4개로 분할한다.

In [21]:
for dataset in full_data:
    dataset['Fare']= dataset['Fare'].fillna(train['Fare'].median())

train['CategoricalFare']= pd.qcut(train['Fare'], 4)
# qcut: 데이터를 나누고싶은 구역에 모두 똑같은 개수가 들어가도록 나눈다.

train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index= False).mean()

,CategoricalFare,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


6. Age: 나이
- 결측치 있음: 평균-표준편차 와 평균+표준편차 사이의 랜덤 값으로 채운다.
- 5개로 분할한다.

In [33]:
for dataset in full_data:
    age_avg= dataset['Age'].mean() # 평균
    age_std= dataset['Age'].std() # 표준편차
    age_null_count= dataset['Age'].isnull().sum() # 결측치 수

    age_null_random_list= np.random.randint(age_avg - age_std,
                                           age_avg + age_std,
                                           size= age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])]= age_null_random_list
    # 결측치가 있는 행만 불러와서 age_null_random_list를 넣는다.
    dataset['Age']= dataset['Age'].astype(int)
    dataset['CategoricalAge']= pd.cut(train['Age'], 5)
    # cut: 데이터 값을 기준으로 일정하게 자른다.

train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index= False).mean()

C:\Users\82109\AppData\Local\Temp\ipykernel_29772\480647783.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'][np.isnan(dataset['Age'])]= age_null_random_list
C:\Users\82109\AppData\Local\Temp\ipykernel_29772\480647783.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'][np.isnan(dataset['Age'])]= age_null_random_list


,CategoricalAge,Survived
0,"(-0.08, 16.0]",0.553571
1,"(16.0, 32.0]",0.347534
2,"(32.0, 48.0]",0.371542
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


7. Name: 이름
- 이름의 호칭을 찾을 수 있다.

In [34]:
import re as re
def get_title(name):
    title_search= re.search('([A-Za-z]+)\.', name)
    # 다른데서는 data.Name.str.extract('([A-Za-z]+)\.') 으로 했다.
    if title_search:
        return title_search.group(1)
        # 만약 호칭이 있다면 group 1에 매칭된 것을 return 하라
    return ''
    # 아니라면 아무것도 return 하지 마라

for dataset in full_data:
    dataset['Title']= dataset['Name'].apply(get_title)

pd.crosstab(train['Title'], train['Sex'])
# crosstab: 범주형 데이터 2개를 비교 분석한다.

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40
